> Enterprise Web C#

# Chapter 9 - Data, the new raw material - Solutions

Before you start, uncomment the correct connection string below.

**For lists: always convert the returned `IEnumerable` to a `List` and use `myList.ForEach(Console.WriteLine)` to print the query result.**

For macOS/Linux: we assume you picked the same password as in our tutorial.

In [ ]:
#r "nuget:Dapper"
#r "nuget:Dapper.Contrib"
#r "nuget:System.Data.SqlClient"

using Dapper;
using System;
using System.Collections.Generic;
using System.Data;
using System.Data.SqlClient;
using System.Linq;

// Windows:
var connectionString = "Server=localhost;Database=Northwind;Trusted_Connection = True;";

// macOS/Linux:
// var connectionString = "Data Source=127.0.0.1,1433;Initial Catalog=Northwind;User ID=sa;Password=Password123!";

1. Fetch all employees
    - Order by first and last name
    - Only return these fields:
        - EmployeeID
        - FirstName
        - LastName
        - Title
        - Country
    - Implement a `ToString` which concatenates the first and last name of an employee.

You should get 9 rows. Expected output:
```
Andrew Fuller
Anne Dodsworth
Janet Leverling
Laura Callahan
Margaret Peacock
Michael Suyama
Nancy Davolio
Robert King
Steven Buchanan
```

In [ ]:
class Employee
{
  public int EmployeeID { get; set; }
  public string FirstName { get; set; }
  public string LastName { get; set; }
  public string Title { get; set; }
  public string Country { get; set; }

  public override string ToString() => $"{FirstName} {LastName}";
}

In [ ]:
using (IDbConnection db = new SqlConnection(connectionString)) {
  var sql = @"
    SELECT EmployeeID, FirstName, LastName, Title, Country
    FROM Employees
    ORDER BY FirstName, LastName;
  ";
  var employees = (await db.QueryAsync<Employee>(sql)).ToList();
  employees.ForEach(Console.WriteLine);
}

2. Fetch the first 10 orders
    - Order by the order date
    - Only return these fields:
        - OrderID
        - OrderDate
        - EmployeeID of both Employees and Orders
        - FirstName
        - LastName
        - Title
        - Country
    - Implement a `ToString` which tells the order id, date and who handled the order.

You should get 10 rows. Expected output:
```
10248 (4/07/1996) handled by Steven Buchanan
10249 (5/07/1996) handled by Michael Suyama
10250 (8/07/1996) handled by Margaret Peacock
10251 (8/07/1996) handled by Janet Leverling
10252 (9/07/1996) handled by Margaret Peacock
10253 (10/07/1996) handled by Janet Leverling
10254 (11/07/1996) handled by Steven Buchanan
10255 (12/07/1996) handled by Anne Dodsworth
10256 (15/07/1996) handled by Janet Leverling
10257 (16/07/1996) handled by Margaret Peacock
```

In [ ]:
class Order
{
  public int OrderID { get; set; }
  public int EmployeeID { get; set; }
  public DateTime OrderDate { get; set; }
  public Employee HandledBy { get; set; }

  public override string ToString() => $"{OrderID} ({OrderDate:d}) handled by {HandledBy}";
}

In [ ]:
using (IDbConnection db = new SqlConnection(connectionString)) {
  var sql = @"
    SELECT TOP(10) *
    FROM [dbo].[Orders] o
      JOIN [dbo].[Employees] e ON e.EmployeeID = o.EmployeeID
    ORDER BY OrderDate;
  ";
  var orders = (await db.QueryAsync<Order, Employee, Order>(
    sql,
    (order, employee) => {
      order.HandledBy = employee;
      return order;
    },
    splitOn: "EmployeeID"
  )).ToList();
  orders.ForEach(Console.WriteLine);
}

3. Fetch all territories with their respective region
    - Fetch every field of both entities
    - Order by region description
    - Implement a `ToString` for a region which returns the description (trimmed at the end).
    - Implement a `ToString` for a territory which returns the region description and territory description (trimmed at the end).

Expected output:
```
Eastern Westboro
Eastern Bedford
Eastern Georgetow
Eastern Boston
Eastern Cambridge
Eastern Braintree
Eastern Providence
Eastern Wilton
Eastern Morristown
Eastern Edison
Eastern New York
Eastern New York
Eastern Mellvile
// and many more...
```

In [ ]:
class Region
{
  public int RegionID { get; set; }
  public string RegionDescription { get; set; }
  public override string ToString() => RegionDescription.TrimEnd();
}

class Territory
{
  public int TerritoryID { get; set; }
  public string TerritoryDescription { get; set; }
  public int RegionID { get; set; }
  public Region Region { get; set; }

  public override string ToString() => $"{Region} {TerritoryDescription.TrimEnd()}";
}

In [ ]:
using (IDbConnection db = new SqlConnection(connectionString))
{
  var sql = @"
      SELECT *
      FROM [dbo].[Territories] AS t
        JOIN [dbo].[Region] AS r ON [t].[RegionID] = [r].[RegionID]
      ORDER BY RegionDescription;
    ";
  var orders = (await db.QueryAsync<Territory, Region, Territory>(
    sql,
    (territory, region) => {
        territory.Region = region;
        return territory;
    },
    splitOn: "RegionID"
  )).ToList();
  orders.ForEach(Console.WriteLine);
}

4. Count the number of orders in the database.

Expected output:
```
There are 830 orders
```

In [ ]:
using (IDbConnection db = new SqlConnection(connectionString))
{
  var sql = @"
      SELECT COUNT(*)
      FROM [dbo].[Orders];
    ";
  var nrOfOrders = await db.QueryFirstAsync<int>(sql);
  Console.WriteLine($"There are {nrOfOrders} orders");
}